In [ ]:
## Activate virtual env
# $ !python3 -m virtualenv .venv
# $ !source .venv/bin/activate

## Install OpenAI package
# $ !pip install openai

## Export Gilas.io API key
# $ os.environ["GILAS_API_KEY"]='...'

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  

client = OpenAI(
    api_key=os.environ.get("GILAS_API_KEY"),
    base_url="https://api.gilas.io/v1/"
)

In [ ]:
def get_embedding(text, dimentions=1531, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], dimensions=dimentions, model=model).data[0].embedding

In [ ]:
import pandas as pd
import ast

# load dataset
input_datapath = "../data/reviews_with_embeddings_30.csv"
df = pd.read_csv(input_datapath, index_col=0, delimiter=";")
df['embedding'] = df['embedding'].apply(lambda x: ast.literal_eval(x))


In [ ]:
from scipy import spatial  # for calculating vector similarities for search

# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 10
) -> tuple[list[str], list[float]]:
    
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding = get_embedding(query, dimentions=100) # dimentions=100 should be aligned with the dimentions of the original data
    strings_and_relatednesses = [
        (row["Text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [ ]:
# example
strings, relatednesses = strings_ranked_by_relatedness("What flavors are in McCann's Instant Irish Oatmeal variety pack?", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

In [ ]:
# example
strings, relatednesses = strings_ranked_by_relatedness("How to train a large language model?", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)